In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, Flatten
import openai
import matplotlib.image as img

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory ='C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    color_mode = 'rgb',
    shuffle = True , 
    label_mode="categorical" , 
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    directory ='C:/Users/User/Allergy-Pal/kaggle/input/food41/images',
    image_size = (224, 224),
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    color_mode = 'rgb',
    label_mode="categorical" , 
    batch_size=32
)

In [ ]:
classes = []
with open("C:/Users/User/Allergy-Pal/kaggle/input/food41/meta/meta/classes.txt") as f:
    for line in f.readlines():
        classes.append(line.split("\n")[0]) 

print(classes)

In [ ]:
img_size = (180 , 180)
batch_size = 32

callback_list=[
    callbacks.EarlyStopping(monitor="val_accuracy",patience=10,restore_best_weights=True),
    callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_accuracy",patience=3)
]

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top = False , input_shape=(224 , 224 , 3))  

base_model.trainable = False 

inputs = tf.keras.layers.Input(shape = (224 , 224 , 3) , name = "Input_layer")
x = base_model(inputs)
x = tf.keras.layers.Dense(101)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(101 , 
                                activation = "softmax" , 
                                name = "Output_layer")(x) 
Model_1 = tf.keras.Model(inputs , outputs) 

Model_1.compile(
    loss = tf.keras.losses.categorical_crossentropy ,
    optimizer = tf.keras.optimizers.Adam() , 
    metrics = ['accuracy'] 
)

Model_1_History = Model_1.fit(train_ds 
                              , validation_data = val_ds ,
                              epochs = 6  ,
                              verbose = 1 ,
                              callbacks = callback_list)

In [ ]:
Model_1 = tf.keras.models.load_model("C:/Users/User/Allergy-Pal/model.keras")

base_model.trainable = True

for layer in base_model.layers[:-10]:
    layer.trainable = False

Model_1.compile(loss = "categorical_crossentropy" ,
                optimizer = tf.keras.optimizers.Adam(learning_rate= 0.0001), 
                metrics = ["accuracy"]
               )

initial_epoch = 5

Fine_Tune_epoch = initial_epoch + 5

Stage_2_history = Model_1.fit(train_ds ,
                              epochs = Fine_Tune_epoch , 
                              validation_data = val_ds ,
                              validation_steps = len(val_ds) ,
                              initial_epoch = initial_epoch-1)

In [ ]:
Model_1.save("C:/Users/User/Allergy-Pal/model.keras")

img = tf.keras.utils.load_img(
    "C:/Users/User/Downloads/pizza123.jpg", target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) 


predictions = Model_1.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f}% percent confidence."
    .format(classes[np.argmax(score)], 100 * np.max(predictions[0]))
)